<h5 style="text-align:center">Create Adjacency Matrix for Top-k FromFile - ET(3sec)</h5>

In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import statistics 
import Algorithm
import warnings
warnings.filterwarnings("ignore")

totalBTS=3006 ###Number Of Total BTS That readed from file  
k2 = [200] ### Initialization Number Of Cluster
k = []

In [3]:
##Standard deviation calculation function for heterogeneous servers mode
def Calc_SD(WLoads,S_Proc): 
    ii=len(WLoads)
    W_hat = 0 
    SD=0
    for k in range(0,ii):
        W_hat+=WLoads[k]/S_Proc[k]; 
    W_hat=W_hat/ii; 
    for k in range(0,ii):
        SD+=(WLoads[k]/S_Proc[k]-W_hat)**2;  
   
    return np.sqrt((SD/ii))

In [4]:
#Function for calculation of network traffic, access delay and standard deviation using the load information of each BTS and the corresponding cluster, which is stored in the file by the corresponding algorithm.
def Calculate(ii,file):
    df = pd.read_csv('BaseData/Data ('+file+'_List).csv') 
    df_HServers = pd.read_csv('BaseData/Start/HServers3.csv').to_numpy()   
    dfs=df.groupby(['SCode'])['WLoad'].sum()
    nettraffic=0
    sculsterWL=[]
    H_index=0
    H_WL = 0
    for i,item in df.groupby(['SCode']):
        clusterWL = item['WLoad'].sum()
        sculsterWL.append(clusterWL)
        for index, row in item.iterrows():
            nettraffic+=(row[2]*row[6])
    sculsterWL.sort(reverse=True)
     
    print("H WorkLoad for",file," is :" , Calc_SD(sculsterWL,df_HServers))
    dfs.to_csv ('BaseData/Data ('+file+'_ListServer).csv', index=None, header = True)  
    cnt = len(sculsterWL)  
    avg= sum(sculsterWL)/len(sculsterWL)
    sm=0
    for i in sculsterWL:
        sm+=(i-avg)**2
    mystd = np.sqrt(sm/cnt)
    SWorkLoad=np.std(dfs)  
  
    with open('BaseData/Data ('+file+'_Result).csv', 'a') as f: 
        f.writelines(','.join([str(x) for x in [ii,df['BCode'].count(),int(nettraffic),(int)(mystd),(df['Distance'].sum())]])+'\n')


In [7]:
#Load calculation for heterogeneous mode
k = k2
df = pd.read_csv('BaseData/Start/Data (02Adjacency_Matrix)-'+str(totalBTS)+'.csv' , header=None)
df2 = pd.read_csv('BaseData/Start/Data (01UniqueBts)-'+str(totalBTS)+'.csv' , names=["BCode", "TUsers", "WLoad", "Lat", "Long"]) 
files = ['Random','TopK','k_Means','RecursiveClustering']
for f in files: 
    with open('BaseData/Data ('+f+'_Result).csv', 'w') as f2:
        f2.writelines('')

for i in k:  
    for j in range (0,1):
        Algorithm.k_Means(i,'k_Means',totalBTS)  
        Calculate(i,'k_Means')  
        Algorithm.Random(df,df2,i,'Random') 
        Calculate(i,'Random') 
    Algorithm.TopK(df,df2,i,'TopK') 
    Calculate(i,'TopK')
    Calculate(i,'RecursiveClustering')

H WorkLoad for k_Means  is : [0.0016137]
H WorkLoad for Random  is : [0.00232688]
H WorkLoad for TopK  is : [0.00139149]
H WorkLoad for RecursiveClustering  is : [0.00025892]


In [8]:
#Calculate the mean value for network traffic, access delay and standard deviation for multiple runs
files = ['Random','TopK','k_Means','RecursiveClustering']
for f in files: 
    df = pd.read_csv('BaseData/Data ('+f+'_Result).csv', names=["K","Bts","NetT","WorkL","Dist"])
    df.drop('Bts', axis=1, inplace=True)
    print(f,df.iloc[0,0],df["NetT"].mean(),df["WorkL"].mean(),df["Dist"].mean())
 

Random 8 181146.0 37.0 7364.0
TopK 8 181530.0 22.0 7887.0
k_Means 8 127425.0 25.0 5548.0
RecursiveClustering 8 132412.0 4.0 7246.0
